In [2]:
import sklearn
import math
import pandas as pd
import seaborn as sns

In [3]:
df = pd.read_csv('combined.csv')

In [1]:
import sys
num_SM = 200
max_shared_memory_per_SM_bytes = 1111
# theoricial numbers
max_threads_per_SM = 2222

iter = []
smem_used_sQ = []
smem_used_sK = []
smem_used_sV = []
used_shared_mem = []
smem_used_sQ_layout = []
smem_used_sK_layout = []
smem_used_sV_layout = []
num_sQ_per_iter = []
num_sK_per_iter = []
num_sV_per_iter = []
tSrQ_per_iter = []
tSrK_per_iter = []
tOrVt_per_iter = []
tSgS_per_iter = []
acc_o_per_iter = []
sVt_per_iter = []
tSgS_per_iter = []
gP_per_iter = []
max_blocks_per_SM_shared_memory_ = []
max_blocks_per_SM_threads_=[]
threads_per_block = []
blocks_per_SM = []
iter_pair = []
threads_per_SM = []
percent_Smem_used_per_sm = []
work_amount_per_thread = []
total_threads = []

for i in range(len(df['num_heads'])):
    # Grid dimensions
    grid_x = df['num_m_blocks'][i] 
    grid_y = df['batch_size'][i]
    grid_z = df['num_heads'][i]  # Number of heads (params.h) 528 

    # Total number of blocks in the grid
    total_blocks_in_grid = grid_x * grid_y * grid_z

    num_threads_used_per_block = 32*df['Nwarps'][i]
    threads_per_block.append(num_threads_used_per_block)
    sQ = df['kBlockM'][i]*df['headdim'][i]*2 
    sKV = df['kBlockN'][i]*df['headdim'][i]*2 
    kSmemSize = sQ + (2*sKV) 
    print(max_shared_memory_per_SM_bytes, kSmemSize)
    max_blocks_per_SM_shared_memory = math.floor(237568 / kSmemSize) # in Bytes
    max_blocks_per_SM_shared_memory_.append(max_blocks_per_SM_shared_memory)
    max_blocks_per_SM_threads = math.floor(2048/num_threads_used_per_block)
    max_blocks_per_SM_threads_.append(max_blocks_per_SM_threads)
    # Determine the limiting factor
    max_blocks_per_SM = min(max_blocks_per_SM_shared_memory, max_blocks_per_SM_threads)
    blocks_per_SM.append(max_blocks_per_SM)
    threads_per_SM_ = max_blocks_per_SM*num_threads_used_per_block
    threads_per_SM.append(threads_per_SM_)
   
    # Total number of concurrent blocks
    total_concurrent_blocks = max_blocks_per_SM * num_SM


    # Calculate number of waves needed on each GPU
    number_of_iterations = math.ceil(total_blocks_in_grid / total_concurrent_blocks)
    work_per_thread = total_blocks_in_grid/threads_per_SM_
    work_amount_per_thread.append(work_per_thread)
    iter_pair.append([total_blocks_in_grid, total_concurrent_blocks, max_blocks_per_SM, max_blocks_per_SM_shared_memory, max_blocks_per_SM_threads])

    iter.append(number_of_iterations)
     # Q
    actual_list_q = eval(df['sQ'][i])
    # sQ_size = actual_list_q[0]*actual_list_q[1]
    smem_used_sQ.append(sQ)
    smem_used_sQ_layout.append(int(str(actual_list_q[0])+str(actual_list_q[1])))
    actual_list_tSrQ = eval(df['tSrQ'][i])
    tSrQ_per_iter.append(int(str(actual_list_tSrQ[0])+str(actual_list_tSrQ[1])))
    mQ_size = df['headdim'][i]*df['num_heads'][i]*df['seqlen'][i]
    num_sQ_per_iter.append((mQ_size/sQ))
    # K
    actual_list_k = eval(df['sK'][i])
    # sK_size = actual_list_k[0]*actual_list_k[1]
    smem_used_sK.append(sKV)
    smem_used_sK_layout.append(int(str(actual_list_k[0])+str(actual_list_k[1])))
    actual_list_tSrK = eval(df['tSrK'][i])
    tSrK_per_iter.append(int(str(actual_list_tSrK[0])+str(actual_list_tSrK[1])))
    mK_size = df['headdim'][i]*df['num_heads'][i]*df['seqlen'][i]
    num_sK_per_iter.append((mK_size/sKV))
    percent_Smem_used_per_sm.append(kSmemSize/237568)
    # V
    actual_list_v = eval(df['sV'][i])
    # sV_size = actual_list_v[0]*actual_list_v[1]
    smem_used_sV.append(sKV)
    smem_used_sV_layout.append(int(str(actual_list_v[0])+str(actual_list_v[1])))
    mV_size = df['headdim'][i]*df['num_heads'][i]*df['seqlen'][i]
    actual_list_tOrVt = eval(df['tOrVt'][i])
    tOrVt_per_iter.append(int(str(actual_list_tOrVt[0])+str(actual_list_tOrVt[1])))
    actual_list_acc_o = eval(df['acc_o'][i])
    acc_o_per_iter.append(int(str(actual_list_acc_o[0])+str(actual_list_acc_o[1])))
    actual_list_sVt = eval(df['sVt'][i])
    sVt_per_iter.append(int(str(actual_list_sVt[0])+str(actual_list_sVt[1])))
    actual_list_tSgS = eval(df['tSgS'][i])
    tSgS_per_iter.append(int(str(actual_list_tSgS[0])+str(actual_list_tSgS[1])))
    actual_list_gP = eval(df['gP'][i])
    gP_per_iter.append(int(str(actual_list_gP[0])+str(actual_list_gP[1])))
    num_sV_per_iter.append((mV_size/sKV))
    used_shared_mem.append(kSmemSize)
    








In [193]:
df['iter_wave_per_kernel_call'] = iter
df['shared_mem_used_by_SQ_SK_SV'] = used_shared_mem
df['sQ'] = smem_used_sQ_layout
df['sK'] = smem_used_sK_layout
df['sV'] = smem_used_sV_layout
df['num_blocks_sQ_per_iter'] = num_sQ_per_iter
df['num_blocks_sK_per_iter'] = num_sV_per_iter
df['num_blocks_sV_per_iter'] = num_sK_per_iter
df['tSrQ_layout'] = tSrQ_per_iter
df['tOrVt_layout'] = tOrVt_per_iter
df['tSrK_layout'] = tSrK_per_iter
df['acc_o_layout'] = acc_o_per_iter
df['sVt_layout'] = sVt_per_iter
df['tSgS_layout'] = tSgS_per_iter
df['gP_layout'] = gP_per_iter
df['threads_per_block'] = threads_per_block
df['max_blocks_per_SM_shared_memory'] = max_blocks_per_SM_shared_memory_
df['max_blocks_per_SM_threads_'] = max_blocks_per_SM_threads_
df['blocks_per_SM'] = blocks_per_SM
df['threads_per_SM'] = threads_per_SM
df['percent_Smem_used_per_sm'] = percent_Smem_used_per_sm

In [196]:
df = df.drop([ 'acc_o', 'tSrQ', 'tOrVt', 'tSrK', 'sVt', 'tSgS', 'gP', 'gV', 'gK', 'gQ'], axis=1)


In [11]:
# df.columns

In [201]:
# delete pytorch ones
df = df[df['method'] != 'Pytorch']

In [5]:
# No casual mask and dropouts
df_1 = df

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

# Separate the target from the features
X = df_1.drop(columns=['fwd_TFLOPs/s'])
y = df_1['fwd_TFLOPs/s']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.000001, random_state=42)

df['Nwarps'].unique()
X_test['Nwarps'].unique()
# Initialize the RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Feature importance
feature_importances = model.feature_importances_

# Create a DataFrame for feature importance
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Feature Importance in RandomForest Regressor')
plt.gca().invert_yaxis()
plt.show()

# Display the feature importance dataframe
# import ace_tools as tools; tools.display_dataframe_to_user(name="Feature Importance", dataframe=feature_importance_df)


In [6]:
from sklearn.inspection import PartialDependenceDisplay
# Partial Dependence Plots
most_important_features = feature_importance_df['Feature'].head(20).tolist()


# Plotting partial dependence for the top 2 features
fig, ax = plt.subplots(figsize=(40, 24))
PartialDependenceDisplay.from_estimator(model, X_train, features=most_important_features, ax=ax)
plt.suptitle('Partial Dependence Plots for Top Features')
plt.subplots_adjust(top=0.9)  # Adjust layout to make space for the title
plt.show()

#  Plotting partial dependence for the top 2 features in a grid
fig, ax = plt.subplots(figsize=(20, 12))

print(PartialDependenceDisplay.from_estimator(model, X_train, features=[['kBlockM', 'percent_Smem_used_per_sm']], kind='average', ax=ax))
# Adjust layout to make space for everything
plt.suptitle('Partial Dependence Plots for Top 2 Features', fontsize=16)
# plt.subplots_adjust(left=0.1, bottom=1, right=0.9, top=0.85, wspace=0.2, hspace=0.2)
plt.show()


In [7]:

fig, ax = plt.subplots(figsize=(20, 12))
print(PartialDependenceDisplay.from_estimator(model, X_train, features=[['max_blocks_per_SM_threads', 'Nwarps']], kind='average', ax=ax))
# Adjust layout to make space for everything
plt.suptitle('Partial Dependence Plots for Top 2 Features', fontsize=16)
plt.subplots_adjust(left=0.1, bottom=1, right=0.9, top=0.85, wspace=0.2, hspace=0.2)
plt.show()

In [8]:

fig, ax = plt.subplots(figsize=(20, 12))
print(PartialDependenceDisplay.from_estimator(model, X_train, features=[['threads_per_block', 'Nwarps']], kind='average', ax=ax))
# Adjust layout to make space for everything
plt.suptitle('Partial Dependence Plots for Top 2 Features', fontsize=16)
plt.subplots_adjust(left=0.1, bottom=1, right=0.9, top=0.85, wspace=0.2, hspace=0.2)
plt.show()

In [9]:

fig, ax = plt.subplots(figsize=(20, 12))
print(PartialDependenceDisplay.from_estimator(model, X_train, features=[['blocks_per_SM', 'Nwarps']], kind='average', ax=ax))
# Adjust layout to make space for everything
plt.suptitle('Partial Dependence Plots for Top 2 Features', fontsize=16)
plt.subplots_adjust(left=0.1, bottom=1, right=0.9, top=0.85, wspace=0.2, hspace=0.2)
plt.show()